# CrewAI Knowledge - Tutorial Completo

## ¿Qué es Knowledge en CrewAI?

Knowledge en CrewAI es un sistema poderoso que permite a los agentes de IA acceder y utilizar fuentes de información externas durante sus tareas. Piensa en ello como darle a tus agentes una biblioteca de referencia que pueden consultar mientras trabajan.

### Beneficios principales:
- **Mejorar agentes** con información específica del dominio
- **Apoyar decisiones** con datos del mundo real
- **Mantener contexto** a través de conversaciones
- **Fundamentar respuestas** en información factual

## Configuración Inicial

Para fuentes de Knowledge basadas en archivos, asegúrate de colocar tus archivos en un directorio `knowledge` en la raíz de tu proyecto. También usa rutas relativas desde el directorio `knowledge` cuando crees la fuente.

In [1]:
# Instalación de dependencias
!pip install crewai
!pip install docling  # Para contenido web

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


## 1. Knowledge Básico con String

Ejemplo más simple usando una cadena de texto como fuente de conocimiento.

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Crear una fuente de conocimiento
content = "El nombre del usuario es Juan. Tiene 30 años y vive en San Francisco."
string_source = StringKnowledgeSource(content=content)

# Crear un LLM con temperatura 0 para salidas determinísticas
llm = LLM(model="gpt-4o-mini", temperature=0)

# Crear un agente con el almacén de conocimiento
agent = Agent(
    role="Sobre el Usuario",
    goal="Sabes todo sobre el usuario.",
    backstory="Eres un maestro en entender personas y sus preferencias.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

task = Task(
    description="Responde las siguientes preguntas sobre el usuario: {question}",
    expected_output="Una respuesta a la pregunta.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source],  # Habilitar knowledge agregando las fuentes aquí
)

result = crew.kickoff(inputs={"question": "¿En qué ciudad vive Juan y cuántos años tiene?"})
print(result)

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

## 2. Knowledge con Contenido Web

Ejemplo usando contenido web como fuente de conocimiento.

In [3]:
from crewai import LLM, Agent, Crew, Process, Task
from crewai.knowledge.source.crew_docling_source import CrewDoclingSource

# Crear una fuente de conocimiento desde contenido web
content_source = CrewDoclingSource(
    file_paths=[
        "https://lilianweng.github.io/posts/2024-11-28-reward-hacking",
        "https://lilianweng.github.io/posts/2024-07-07-hallucination",
    ],
)

# Crear un LLM con temperatura 0 para salidas determinísticas
llm = LLM(model="gpt-4o-mini", temperature=0)

# Crear un agente con el almacén de conocimiento
agent = Agent(
    role="Sobre Papers",
    goal="Sabes todo sobre los papers.",
    backstory="Eres un maestro en entender papers y su contenido.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

task = Task(
    description="Responde las siguientes preguntas sobre los papers: {question}",
    expected_output="Una respuesta a la pregunta.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[content_source],
)

result = crew.kickoff(
    inputs={"question": "¿De qué trata el paper sobre reward hacking? Asegúrate de proporcionar fuentes."}
)
print(result)

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

## 3. Fuentes de Knowledge Soportadas

CrewAI soporta varios tipos de fuentes de conocimiento:

In [ ]:
# Text File Knowledge Source
from crewai.knowledge.source.text_file_knowledge_source import TextFileKnowledgeSource

text_source = TextFileKnowledgeSource(
    file_paths=["document.txt", "another.txt"]
)

# PDF Knowledge Source
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

pdf_source = PDFKnowledgeSource(
    file_paths=["document.pdf", "another.pdf"]
)

# CSV Knowledge Source
from crewai.knowledge.source.csv_knowledge_source import CSVKnowledgeSource

csv_source = CSVKnowledgeSource(
    file_paths=["data.csv"]
)

# Excel Knowledge Source
from crewai.knowledge.source.excel_knowledge_source import ExcelKnowledgeSource

excel_source = ExcelKnowledgeSource(
    file_paths=["spreadsheet.xlsx"]
)

# JSON Knowledge Source
from crewai.knowledge.source.json_knowledge_source import JSONKnowledgeSource

json_source = JSONKnowledgeSource(
    file_paths=["data.json"]
)

## 4. Knowledge a Nivel de Agente vs Crew

### Knowledge a Nivel de Agente (Independiente)

In [ ]:
from crewai import Agent, Task, Crew
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Agente con su propio conocimiento - NO necesita conocimiento del crew
specialist_knowledge = StringKnowledgeSource(
    content="Información técnica especializada solo para este agente"
)

specialist_agent = Agent(
    role="Especialista Técnico",
    goal="Proporcionar experiencia técnica",
    backstory="Experto en dominios técnicos especializados",
    knowledge_sources=[specialist_knowledge]  # Conocimiento específico del agente
)

task = Task(
    description="Responde preguntas técnicas",
    agent=specialist_agent,
    expected_output="Respuesta técnica"
)

# No se requiere conocimiento a nivel de crew
crew = Crew(
    agents=[specialist_agent],
    tasks=[task]
)

result = crew.kickoff()  # El conocimiento del agente funciona independientemente
print(result)

### Knowledge a Nivel de Crew (Compartido)

In [ ]:
# Conocimiento del crew (compartido por todos los agentes)
crew_knowledge = StringKnowledgeSource(
    content="Políticas de la empresa e información general para todos los agentes"
)

# Conocimiento específico del agente
specialist_knowledge = StringKnowledgeSource(
    content="Especificaciones técnicas que solo necesita el especialista"
)

specialist = Agent(
    role="Especialista Técnico",
    goal="Proporcionar experiencia técnica",
    backstory="Experto técnico",
    knowledge_sources=[specialist_knowledge]  # Específico del agente
)

generalist = Agent(
    role="Asistente General",
    goal="Proporcionar asistencia general",
    backstory="Ayudante general"
    # Sin conocimiento específico del agente
)

crew = Crew(
    agents=[specialist, generalist],
    tasks=[...],
    knowledge_sources=[crew_knowledge]  # Conocimiento del crew
)

# Resultado:
# - specialist obtiene: crew_knowledge + specialist_knowledge
# - generalist obtiene: crew_knowledge solamente

## 5. Ejemplo Completo: Múltiples Agentes con Diferente Knowledge

In [ ]:
# Diferente conocimiento para diferentes agentes
sales_knowledge = StringKnowledgeSource(content="Procedimientos de ventas y precios")
tech_knowledge = StringKnowledgeSource(content="Documentación técnica")
support_knowledge = StringKnowledgeSource(content="Procedimientos de soporte")

# Agente de ventas
sales_agent = Agent(
    role="Agente de Ventas",
    goal="Vender productos y servicios",
    backstory="Experto en ventas",
    knowledge_sources=[sales_knowledge]
)

# Agente técnico
tech_agent = Agent(
    role="Agente Técnico",
    goal="Resolver problemas técnicos",
    backstory="Especialista técnico",
    knowledge_sources=[tech_knowledge]
)

# Agente de soporte
support_agent = Agent(
    role="Agente de Soporte",
    goal="Proporcionar soporte al cliente",
    backstory="Especialista en soporte",
    knowledge_sources=[support_knowledge]
)

# Crew con conocimiento compartido
company_knowledge = StringKnowledgeSource(content="Información general de la empresa")

crew = Crew(
    agents=[sales_agent, tech_agent, support_agent],
    tasks=[...],
    knowledge_sources=[company_knowledge]
)

## 6. Debugging y Troubleshooting

### Verificar Inicialización del Knowledge del Agente

In [ ]:
from crewai import Agent, Crew, Task
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

knowledge_source = StringKnowledgeSource(content="Test knowledge")
agent = Agent(
    role="Test Agent",
    goal="Test knowledge",
    backstory="Testing",
    knowledge_sources=[knowledge_source]
)
crew = Crew(agents=[agent], tasks=[Task(...)])

# Antes del kickoff - knowledge no inicializado
print(f"Antes del kickoff - Agent knowledge: {getattr(agent, 'knowledge', None)}")

crew.kickoff()

# Después del kickoff - knowledge inicializado
print(f"Después del kickoff - Agent knowledge: {agent.knowledge}")
print(f"Agent knowledge collection: {agent.knowledge.storage.collection_name}")
print(f"Number of sources: {len(agent.knowledge.sources)}")

### Verificar Ubicaciones de Almacenamiento

In [ ]:
import os
from crewai.utilities.paths import db_storage_path

# Verificar estructura de almacenamiento
storage_path = db_storage_path()
knowledge_path = os.path.join(storage_path, "knowledge")

if os.path.exists(knowledge_path):
    print("Knowledge collections encontradas:")
    for collection in os.listdir(knowledge_path):
        collection_path = os.path.join(knowledge_path, collection)
        if os.path.isdir(collection_path):
            print(f" - {collection}/")
            # Mostrar contenido de la colección
            for item in os.listdir(collection_path):
                print(f" └── {item}")

### Comandos de Reset de Knowledge

In [ ]:
# Reset solo knowledge específico del agente
crew.reset_memories(command_type='agent_knowledge')

# Reset tanto crew como knowledge del agente
crew.reset_memories(command_type='knowledge')

# Comandos CLI
# crewai reset-memories --agent-knowledge  # Solo knowledge del agente
# crewai reset-memories --knowledge        # Todo el knowledge

## 7. Mejores Prácticas

### Configuración de Chunks
- Mantén tamaños de chunks apropiados para tu tipo de contenido
- Considera superposición de contenido para preservar contexto
- Organiza información relacionada en fuentes de conocimiento separadas
- Ajusta tamaños de chunks basado en la complejidad del contenido

### Configuración de Embeddings
- Configura modelos de embedding apropiados
- Considera usar proveedores de embedding locales para procesamiento más rápido
- Con la estructura típica de archivos proporcionada por CrewAI, las fuentes de conocimiento se incrustan cada vez que se activa el kickoff
- Si las fuentes de conocimiento son grandes, esto lleva a ineficiencia y mayor latencia
- Para resolver esto, inicializa directamente el parámetro knowledge en lugar del parámetro knowledge_sources

### Organización del Knowledge
- Usa knowledge a nivel de agente para información específica del rol
- Usa knowledge a nivel de crew para información compartida que todos los agentes necesitan
- Configura embedders a nivel de agente si necesitas diferentes estrategias de embedding
- Usa nombres de colección consistentes manteniendo roles de agentes descriptivos

### Testing y Monitoreo
- Prueba la inicialización del knowledge verificando agent.knowledge después del kickoff
- Monitorea ubicaciones de almacenamiento para entender dónde se almacena el knowledge
- Reset knowledge apropiadamente usando los tipos de comandos correctos

### Producción
- Establece CREWAI_STORAGE_DIR en una ubicación conocida en producción
- Elige proveedores de embedding explícitos para que coincidan con tu configuración de LLM
- Monitorea el tamaño del almacenamiento de knowledge a medida que crece con adiciones de documentos
- Organiza fuentes de conocimiento por dominio o propósito usando nombres de colección
- Incluye directorios de knowledge en tus estrategias de backup y deployment
- Establece permisos de archivo apropiados para archivos de knowledge y directorios de almacenamiento
- Usa variables de entorno para API keys y configuración sensible

## 8. Problemas Comunes y Soluciones

### Errores "File not found"
```python
# Asegúrate de que los archivos estén en la ubicación correcta
from crewai.utilities.constants import KNOWLEDGE_DIRECTORY
import os

knowledge_dir = KNOWLEDGE_DIRECTORY  # Usualmente "knowledge"
file_path = os.path.join(knowledge_dir, "your_file.pdf")

if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Expected knowledge directory: {os.path.abspath(knowledge_dir)}")
```

### Errores "Embedding dimension mismatch"
```python
# Esto sucede cuando cambias proveedores de embedding
# Reset knowledge storage para limpiar embeddings antiguos
crew.reset_memories(command_type='knowledge')

# O usa proveedores de embedding consistentes
crew = Crew(
    agents=[...],
    tasks=[...],
    knowledge_sources=[...],
    embedder={"provider": "openai", "config": {"model": "text-embedding-3-small"}}
)
```

### Errores "ChromaDB permission denied"
```bash
# Arreglar permisos de almacenamiento
chmod -R 755 ~/.local/share/CrewAI/
```

### Knowledge no persiste entre ejecuciones
```python
# Verificar consistencia de ubicación de almacenamiento
import os
from crewai.utilities.paths import db_storage_path

print("CREWAI_STORAGE_DIR:", os.getenv("CREWAI_STORAGE_DIR"))
print("Computed storage path:", db_storage_path())
print("Knowledge path:", os.path.join(db_storage_path(), "knowledge"))
```

## 9. Ejemplo Práctico: Sistema de Soporte con Knowledge

Vamos a crear un sistema de soporte que use diferentes fuentes de conocimiento.

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Crear fuentes de conocimiento
product_knowledge = StringKnowledgeSource(
    content="""
    Producto A: Precio $100, características: alta calidad, garantía 2 años
    Producto B: Precio $50, características: calidad media, garantía 1 año
    Producto C: Precio $200, características: premium, garantía 3 años
    """
)

policy_knowledge = StringKnowledgeSource(
    content="""
    Política de devoluciones: 30 días para productos nuevos
    Política de garantía: Cobertura completa durante el período de garantía
    Política de envío: Gratis para compras superiores a $150
    """
)

technical_knowledge = StringKnowledgeSource(
    content="""
    Solución de problemas comunes:
    - Problema 1: Reiniciar el dispositivo
    - Problema 2: Verificar conexiones
    - Problema 3: Actualizar software
    """
)

# Crear agentes especializados
sales_agent = Agent(
    role="Agente de Ventas",
    goal="Ayudar con información de productos y ventas",
    backstory="Especialista en productos y ventas",
    knowledge_sources=[product_knowledge],
    verbose=True
)

support_agent = Agent(
    role="Agente de Soporte Técnico",
    goal="Resolver problemas técnicos",
    backstory="Especialista en resolución de problemas",
    knowledge_sources=[technical_knowledge],
    verbose=True
)

policy_agent = Agent(
    role="Agente de Políticas",
    goal="Informar sobre políticas de la empresa",
    backstory="Experto en políticas y procedimientos",
    knowledge_sources=[policy_knowledge],
    verbose=True
)

# Crear tareas
sales_task = Task(
    description="Responde preguntas sobre productos: {question}",
    agent=sales_agent,
    expected_output="Información detallada sobre productos"
)

support_task = Task(
    description="Resuelve problemas técnicos: {issue}",
    agent=support_agent,
    expected_output="Solución al problema técnico"
)

policy_task = Task(
    description="Informa sobre políticas: {policy_question}",
    agent=policy_agent,
    expected_output="Información sobre políticas"
)

# Crear crew con conocimiento compartido
company_info = StringKnowledgeSource(
    content="Empresa XYZ: Fundada en 2020, especializada en tecnología"
)

crew = Crew(
    agents=[sales_agent, support_agent, policy_agent],
    tasks=[sales_task, support_task, policy_task],
    knowledge_sources=[company_info],
    verbose=True,
    process=Process.sequential
)

# Ejecutar consultas
questions = [
    {"question": "¿Cuál es el precio del Producto A?"},
    {"issue": "Mi dispositivo no funciona correctamente"},
    {"policy_question": "¿Cuál es la política de devoluciones?"}
]

for i, query in enumerate(questions, 1):
    print(f"\n=== Consulta {i} ===")
    result = crew.kickoff(inputs=query)
    print(f"Resultado: {result}")

## Resumen

En este tutorial hemos cubierto:

1. **Conceptos básicos** de Knowledge en CrewAI
2. **Configuración inicial** y estructura de archivos
3. **Tipos de fuentes** de conocimiento soportadas
4. **Knowledge a nivel de agente vs crew**
5. **Debugging y troubleshooting**
6. **Mejores prácticas** para implementación
7. **Ejemplo práctico** de sistema de soporte

### Recursos Adicionales
- [Documentación oficial de CrewAI Knowledge](https://docs.crewai.com/en/concepts/knowledge.md)
- [GitHub Issue sobre optimización de knowledge](https://github.com/crewAIInc/crewAI/issues/2755)

### Comandos Útiles
```bash
# Reset knowledge
crewai reset-memories --knowledge

# Reset solo agent knowledge
crewai reset-memories --agent-knowledge
```